In [1]:
from dask.distributed import Client

client = Client(n_workers=16)

client


Client Scheduler: tcp://127.0.0.1:45533 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 16.42 GB


In [2]:
from dask import delayed

In [3]:
import os, sys, glob
from dask import bag as db

#throw all the words into one big bag, case sensitive
#b = db.read_text(os.path.join('data', '*.txt'))
#b.take(100) 

#'''
##preprocess the files, put all the content info one file 'out.tmp' (now find a better way)
#outfile = open('out.tmp', 'w')
#'''
filelist = []
for file in glob.glob('./data/*.txt'):
    filelist.append(file)

#    linecount = 1
#    with open(file, 'r', encoding='utf-8-sig') as f:
#        for line in f:
#            words = line.strip().split()
#            for word in words:
#                outfile.write('%s\n' % (word.lower()))

#outfile.close()


In [4]:
import json

#throw all the words into one big bag
b = db.read_text(os.path.join('data', '*.txt'), encoding='utf-8-sig').map(lambda x:x.casefold().strip().split()).flatten()

#generate output
newbag = b.frequencies(sort=True) #sort
dic = dict(newbag.take(40)) #get top 40

with open("sp1.json", "w") as outfile:
    json.dump(dic, outfile)

In [5]:
stop_words = []
with open('stopwords.txt') as f:
    for word in f:
        stop_words.append(word.strip())

def is_stopwords(w):
    return w in stop_words

#no-stop-words-bag. filter out all the stop words
nstpbg = b.remove(is_stopwords)

#generate output
newbag = nstpbg.frequencies(sort=True)
dic = dict(newbag.take(40))

with open("sp2.json", "w") as outfile:
    json.dump(dic, outfile)

In [6]:
#discard all words that have only 1 character
def is_1char(w):
    return len(w) == 1

#n1cbg. discard 1-character words
n1cbg = nstpbg.remove(is_1char)

In [7]:
#strip the punctuation
punc_list=[',', '.', '!', '?', ':', ';', "'"]
def trimp(word):
    if word[0] in punc_list:
        word = word[1:]
    if word[-1] in punc_list:
        word = word[:-1]
    return word

npbg = n1cbg.map(trimp)

#generate output
newbag = npbg.frequencies(sort = True)
dic = dict(newbag.take(40))

with open("sp3.json", "w") as outfile:
    json.dump(dic, outfile)

In [8]:
import math
#import tlz as toolz
from tlz import second

baglist = []
tf_idf_list = []

outdict = {}
for file in filelist:
    #print(file)
    bag = db.read_text(file, encoding='utf-8-sig').map(lambda x:x.strip().split()).flatten().str.casefold()
    bag = bag.remove(is_stopwords).remove(is_1char)
    bag = bag.map(trimp)
    #print(bag.take(10))
    baglist.append(bag)
    
assert(len(baglist)==8)



In [ ]:
#this part takes a thousand years ... ...
outdict = {}

def get_idf(word):
    nt = 0
    for bag in baglist:
        #check if the word exists in this file
        #this is definitely a lame implementation, while I'm not able to find a better way to do it
        
        newbag = bag.map(lambda x: x == word)
        #only '{False ***}' or 'True ***, False ***'
        if newbag.distinct().count().compute()>1:
            nt += 1
    if nt == 0:
        # nt should only equal or larger than 1
        assert(nt !=0), "word = %s" % (word)
        return 0
    else:
        return math.log(8/nt)

for bag in baglist:
    tfbag = bag.frequencies(sort = False)
    tf_idf_bag = tfbag.map(lambda x:(x[0], x[1]*get_idf(x[0])))
    #print('get tf_idf')
    #sort by value
    tf_idf_bag = tf_idf_bag.map_partitions(sorted, key=second, reverse = True)
    #print('tf_idf sorted')
    #tf_idf_list.append(tf_idf_bag) <-- no longer needed. already add to outdict
    
    outdict={**outdict, **dict(tf_idf_bag.take(5))}

with open("sp4.json", "w") as outfile:
    json.dump(outdict, outfile)

In [ ]:
# test code
'''
diction = {}
from tlz import second
testb = db.from_sequence(['ef','aaa','aaa','c','d','a', 'd'])
testb = testb.frequencies(sort = False)
testb = testb.map(lambda x:(x[0], x[1]*1.5))
testb = testb.map_partitions(sorted, key=second, reverse = True)
print(dict(testb))
#print(testb.compute())
#diction = Merge(diction, dict(testb))
diction={**diction, **(dict(testb))}
print(diction)
'''

In [ ]:
'''
init = 0
word = 'f'
def count(x, y):
    return (x==word or y == word)
testb = db.from_sequence(['a', 'b', 'c', 'd', 'a'])
print(testb.fold(count).compute())
print(testb.count().compute())
'''

In [ ]:
#print(len(outdict.keys()))